In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
matplotlib.style.use('fivethirtyeight')
matplotlib.style.use('seaborn-talk')
import os
import collections

In [35]:
preDouble = "\\\\iobsdc01\\SharedDocs\\SEER_MEDICARE\\SEER_MEDICARE_STAGING"
new = os.chdir(preDouble)
print(preDouble)

\\iobsdc01\SharedDocs\SEER_MEDICARE\SEER_MEDICARE_STAGING


In [36]:
import pickle
fileObject = open('pure_control_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
controlindices = list(b)

In [37]:
import pickle
fileObject = open('pure_treatment_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
treatmentindices = list(b)

In [38]:
dg = pd.read_pickle('goodindices_medpar.pickle')

In [39]:
dg.set_index('patient_id',inplace=True,drop=False)

In [40]:
def make_icd9sg_dataframe():
    """Make a dataframe that includes the icd9sg descriptins."""
    
    dficd9sg = pd.read_excel('CMS32_DESC_LONG_SHORT_SG.xlsx',
                        convert_float=False,
                        converters = {'PROCEDURE CODE': str})
    dficd9sg = dficd9sg.rename(columns={'PROCEDURE CODE': 'code',
                        'LONG DESCRIPTION': 'long_description',
                        'SHORT DESCRIPTION': 'short_description'})

    return dficd9sg

In [41]:
def make_icd9dx_dataframe():
    """Make a dataframe that includes the icd9dx descriptions."""
    
    dficd9dx = pd.read_excel('CMS32_DESC_LONG_SHORT_DX.xlsx',
                        convert_float=False,
                        converters = {'PROCEDURE CODE': str})
    dficd9dx = dficd9dx.rename(columns={'DIAGNOSIS CODE': 'code',
                                    'LONG DESCRIPTION': 'long_description',
                        'SHORT DESCRIPTION': 'short_description'})

    return dficd9dx

In [42]:

dficd9dx = make_icd9dx_dataframe()

In [43]:
dfcodes = dficd9dx.copy()
dfcodes = dfcodes.set_index('code',drop=False)
dfcodes

,code,long_description,short_description
code,,,
0010,0010,Cholera due to vibrio cholerae,Cholera d/t vib cholerae
0011,0011,Cholera due to vibrio cholerae el tor,Cholera d/t vib el tor
0019,0019,"Cholera, unspecified",Cholera NOS
0020,0020,Typhoid fever,Typhoid fever
0021,0021,Paratyphoid fever A,Paratyphoid fever a
0022,0022,Paratyphoid fever B,Paratyphoid fever b
0023,0023,Paratyphoid fever C,Paratyphoid fever c
0029,0029,"Paratyphoid fever, unspecified",Paratyphoid fever NOS
0030,0030,Salmonella gastroenteritis,Salmonella enteritis


In [44]:
code_dict = dfcodes.long_description.to_dict()
#code_dict

In [45]:
dficd9sg = make_icd9sg_dataframe()


In [46]:
dficd9sg = dficd9sg.set_index('code',drop=False)
dficd9sg


,code,long_description,short_description
code,,,
0001,0001,Therapeutic ultrasound of vessels of head and ...,Ther ult head & neck ves
0002,0002,Therapeutic ultrasound of heart,Ther ultrasound of heart
0003,0003,Therapeutic ultrasound of peripheral vascular ...,Ther ult peripheral ves
0009,0009,Other therapeutic ultrasound,Other therapeutic ultsnd
0010,0010,Implantation of chemotherapeutic agent,Implant chemothera agent
0011,0011,Infusion of drotrecogin alfa (activated),Infus drotrecogin alfa
0012,0012,Administration of inhaled nitric oxide,Adm inhal nitric oxide
0013,0013,Injection or infusion of nesiritide,Inject/infus nesiritide
0014,0014,Injection or infusion of oxazolidinone class o...,Injection oxazolidinone


In [47]:
proc_code_dict = dficd9sg.long_description.to_dict()

In [48]:

dg.head()


,registry,patient_id,age,stdstate,std_cnty,admday,dschgsta,ghopd,ppsind,provcode,...,obic,drgcode,discdest,outclrcde,primpayr,esrdcond,admsrce,admtype,admdxcde,deathadm
patient_id,,,,,,,,,,,,,,,,,,,,,
8869027355,88,8869027355,74,5,430,3,A,NaN,2,NaN,...,A,90,1,0,N,0,1,2,486,0
8869027355,88,8869027355,74,5,430,4,A,NaN,2,NaN,...,A,143,1,0,N,0,7,2,78659,0
8828037361,88,8828037361,69,5,400,4,A,NaN,2,NaN,...,TA,335,1,0,NaN,0,1,3,185,0
8863055354,88,8863055354,76,5,430,6,A,NaN,2,NaN,...,A,143,1,0,NaN,0,7,2,78650,0
2693087051,26,2693087051,72,46,170,6,A,NaN,2,NaN,...,A,334,1,0,NaN,0,1,3,185,0


In [49]:
goodcols = ['patient_id','numdxcde','dgn_cd1','dgn_cd2','dgn_cd3',
            'dgn_cd4','dgn_cd5',
            'dgn_cd6','dgn_cd7','dgn_cd8','dgn_cd9',
            'dgn_cd10','dgn_cd11','dgn_cd12','dgn_cd13',
            'dgn_cd14','dgn_cd15','dgn_cd16','dgn_cd17',
            'dgn_cd18','dgn_cd19','dgn_cd20','dgn_cd21',
            'dgn_cd22','dgn_cd23','dgn_cd24','dgn_cd25',
            'srgcde1','srgcde2','srgcde3','srgcde4','srgcde5',
            'srgcde6','srgcde7','srgcde8','srgcde9','srgcde10',
            'srgcde11','srgcde12','srgcde13','srgcde14','srgcde15',
            'srgcde16','srgcde17','srgcde18','srgcde19','srgcde20',
            'srgcde21','srgcde22','srgcde23','srgcde24','srgcde25',
            'admdxcde','medpar_admission_date']
dg[goodcols].head()

,patient_id,numdxcde,dgn_cd1,dgn_cd2,dgn_cd3,dgn_cd4,dgn_cd5,dgn_cd6,dgn_cd7,dgn_cd8,...,srgcde18,srgcde19,srgcde20,srgcde21,srgcde22,srgcde23,srgcde24,srgcde25,admdxcde,medpar_admission_date
patient_id,,,,,,,,,,,,,,,,,,,,,
8869027355,8869027355,5,486,4659,41401,4019,69018,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,486,4032006
8869027355,8869027355,9,78659,4139,4240,4241,4242,41401,43330,4019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78659,5302006
8828037361,8828037361,3,185,4019,60011,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185,12052006
8863055354,8863055354,6,78650,2875,2720,25000,44389,3669,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78650,12142006
2693087051,2693087051,5,185,78039,4019,2720,3659,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185,2022006


In [50]:
dh = dg[goodcols].drop_duplicates().copy()

In [51]:
dh['dgn_cd1_description'] = dh['dgn_cd1'].astype('str').replace(code_dict)
dh['dgn_cd2_description'] = dh['dgn_cd2'].astype('str').replace(code_dict)
dh['dgn_cd3_description'] = dh['dgn_cd3'].astype('str').replace(code_dict)
dh['dgn_cd4_description'] = dh['dgn_cd4'].astype('str').replace(code_dict)
dh['dgn_cd5_description'] = dh['dgn_cd5'].astype('str').replace(code_dict)

dh['dgn_cd6_description'] = dh['dgn_cd6'].astype('str').replace(code_dict)
dh['dgn_cd7_description'] = dh['dgn_cd7'].astype('str').replace(code_dict)
dh['dgn_cd8_description'] = dh['dgn_cd8'].astype('str').replace(code_dict)
dh['dgn_cd9_description'] = dh['dgn_cd9'].astype('str').replace(code_dict)
dh['dgn_cd10_description'] = dh['dgn_cd10'].astype('str').replace(code_dict)
dh['dgn_cd11_description'] = dh['dgn_cd11'].astype('str').replace(code_dict)
dh['dgn_cd12_description'] = dh['dgn_cd12'].astype('str').replace(code_dict)
dh['dgn_cd13_description'] = dh['dgn_cd13'].astype('str').replace(code_dict)
dh['dgn_cd14_description'] = dh['dgn_cd14'].astype('str').replace(code_dict)
dh['dgn_cd15_description'] = dh['dgn_cd15'].astype('str').replace(code_dict)
dh['dgn_cd16_description'] = dh['dgn_cd16'].astype('str').replace(code_dict)
dh['dgn_cd17_description'] = dh['dgn_cd17'].astype('str').replace(code_dict)
dh['dgn_cd18_description'] = dh['dgn_cd18'].astype('str').replace(code_dict)
dh['dgn_cd19_description'] = dh['dgn_cd19'].astype('str').replace(code_dict)
dh['dgn_cd20_description'] = dh['dgn_cd20'].astype('str').replace(code_dict)

dh['dgn_cd21_description'] = dh['dgn_cd21'].astype('str').replace(code_dict)
dh['dgn_cd22_description'] = dh['dgn_cd22'].astype('str').replace(code_dict)
dh['dgn_cd23_description'] = dh['dgn_cd23'].astype('str').replace(code_dict)
dh['dgn_cd24_description'] = dh['dgn_cd24'].astype('str').replace(code_dict)
dh['dgn_cd25_description'] = dh['dgn_cd25'].astype('str').replace(code_dict)

In [52]:

dh['srgcde1_description'] = dh['srgcde1'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde2_description'] = dh['srgcde2'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde3_description'] = dh['srgcde3'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde4_description'] = dh['srgcde4'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde5_description'] = dh['srgcde5'].fillna(0).astype('int').astype('str').replace(proc_code_dict)


dh['srgcde6_description'] = dh['srgcde6'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde7_description'] = dh['srgcde7'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde8_description'] = dh['srgcde8'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde9_description'] = dh['srgcde9'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde10_description'] = dh['srgcde10'].fillna(0).astype('int').astype('str').replace(proc_code_dict)


dh['srgcde11_description'] = dh['srgcde11'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde12_description'] = dh['srgcde12'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde13_description'] = dh['srgcde13'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde14_description'] = dh['srgcde14'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde15_description'] = dh['srgcde15'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde16_description'] = dh['srgcde16'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde17_description'] = dh['srgcde17'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde18_description'] = dh['srgcde18'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde19_description'] = dh['srgcde19'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde20_description'] = dh['srgcde20'].fillna(0).astype('int').astype('str').replace(proc_code_dict)


dh['srgcde21_description'] = dh['srgcde21'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde22_description'] = dh['srgcde22'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde23_description'] = dh['srgcde23'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde24_description'] = dh['srgcde24'].fillna(0).astype('int').astype('str').replace(proc_code_dict)
dh['srgcde25_description'] = dh['srgcde25'].fillna(0).astype('int').astype('str').replace(proc_code_dict)

In [53]:
dh['admdxcde_description'] = dh['admdxcde'].astype('str').replace(code_dict)

In [54]:
df = pd.read_pickle('dfpedsfFirsts.pickle')

# <font color="#E56717">Complications in the Hospital Claims data</font>

* <span style="font-size:1.5em;">MedPAR: Medicare Provider and Review</span>
* <span style="font-size:1.5em;">One record per stay</span>
* <span style="font-size:1.5em;">Key elements: admission and discharge diagnoses and dates, procedures and procedure dates, revenue center codes for ER</span>

In [55]:
df = df.set_index('patient_id',drop=False)

In [56]:
goodindices = controlindices + treatmentindices
print(len(goodindices),len(controlindices),len(treatmentindices))

1662 831 831


In [57]:
df_all_good_indices = df.loc[goodindices].copy()

In [58]:
del df

In [59]:
monthdict = {'Jan': '01',
            'Feb': '02',
            'Mar': '03',
            'Apr': '04',
            'May': '05',
            'Jun': '06',
            'Jul': '07',
            'Aug': '08',
            'Sep': '09',
            'Oct': '10',
            'Nov': '11',
            'Dec': '12'}


df_all_good_indices['month1'] = df_all_good_indices['modx1'].replace(monthdict)

In [60]:
prostate_filter = df_all_good_indices['site1'].str.contains('prostate',case=False,na=False)
to_drop = list(df_all_good_indices[~prostate_filter].site1.index)
print(to_drop)

drop_set = set(to_drop)
t_set = set(d_t_c.keys())

lose_set = t_set.intersection(drop_set)
print(len(lose_set))

for i in list(lose_set):
    del d_t_c[i]

['4482160083', '2215176050', '8859072472', '2310153070', '8802026371', '8757024520', '8831097343', '8878004314', '8827010107', '4408106073', '8797097489', '8835013250', '4339013024', '4304141320', '4326238080', '4269060013', '8848090384', '4268053014', '8797035478', '8841081424', '8868081313', '4339033024', '4222034004', '8735038409', '8716025459', '4462162024', '8821066343', '8858012304', '8845030491', '8841000365', '8888027441', '4313295000', '8825058106', '8888086461', '4473112043', '8714009560', '8870085471', '8819038335', '8885077334', '8865081345', '8815026310', '8889005430', '4454195034', '4247072024', '8851092441', '8897051314', '4295048060', '8852000315', '8875067360', '4315229060']
0


# <font color="#E56717">We need to only keep claims for which the date of admission is later in time than the cancer diagnosis date</font>


<span style="font-size:1.5em;">3 of the treatment-control pairs are missing month of diagnoses; will not use them when investigating relative likelihood of complications in the MedPAR file</span>

<span style="font-size:1.5em;">Likewise, some 970 rows in the MedPAR file had missing admission date information; will not use them either.</span>

$\rightarrow$ <span style="font-size:1.5em;">2696 unique claims for patients in the PBT and RT (Non PBT) groups, with an admission date later than the cancer dx date</span>

In [63]:
goodtreatmentindices = d_t_c.keys()
goodcontrolindices = d_t_c.values()
print(len(goodtreatmentindices), len(goodcontrolindices))

df_all_good_indices = df_all_good_indices.loc[list(goodcontrolindices) + list(goodtreatmentindices)]

df_all_good_indices['newmonth1'] = df_all_good_indices.month1.fillna('01').copy()

#df_all_good_indices['yrdx_plus_modx'] = df_all_good_indices.yrdx1.astype('str') + df_all_good_indices.newmonth1.astype('str')


#df_all_good_indices['yrdx_plus_modx'] = pd.to_datetime(df_all_good_indices['yrdx_plus_modx'], 
#                                                      format='%Y%m')

#dh = dh.set_index('patient_id',drop=False)

#dgtreatment = dh[dh.patient_id.isin(goodtreatmentindices) & prostate_filter].copy()

#dgcontrol = dh[dh.patient_id.isin(goodcontrolindices) & prostate_filter].copy()

857 857


In [68]:
df_all_good_indices.newmonth1.isnull().sum()

0

In [66]:

df_all_good_indices.month1.isnull().sum()

453

In [69]:
df_all_good_indices['yrdx_plus_modx'] = df_all_good_indices.yrdx1.astype('str') + df_all_good_indices.newmonth1.astype('str')

In [70]:
df_all_good_indices['yrdx_plus_modx'].value_counts()

nan01       448
2007.002     30
2008.007     29
2009.003     27
2008.008     27
2008.005     27
2007.011     27
2007.001     26
2007.008     26
2011.001     24
2011.004     24
2007.003     24
2008.010     24
2009.004     23
2010.007     23
2011.003     22
2009.006     22
2007.006     22
2009.011     21
2006.010     21
2006.007     20
2010.006     20
2009.010     20
2011.006     20
2009.001     20
2009.009     19
2008.004     19
2008.011     19
2006.008     19
2010.010     19
           ... 
2006.011     16
2008.012     16
2011.002     15
2010.003     15
2006.009     15
2008.001     14
2006.005     14
2006.006     14
2010.004     14
2011.008     14
2007.009     14
2007.007     13
2011.007     13
2011.005     13
2007.005     13
2006.003     13
2011.009     13
2008.003     12
2009.008     12
2009.007     12
2006.001     12
2007.010     12
2011.011     11
2011.010     11
2011.012     10
2007.012     10
2010.012      9
2010.009      9
2006.012      8
2006.004      8
Name: yrdx_plus_modx, dt

In [31]:
goodtreatmentindices = d_t_c.keys()
goodcontrolindices = d_t_c.values()
print(len(goodtreatmentindices), len(goodcontrolindices))

857 857


# Just including the prostate cancer patients now.

In [32]:
df_all_good_indices = df_all_good_indices.loc[list(goodcontrolindices) + list(goodtreatmentindices)]

#df_all_good_indices[~prostate_filter].sum() # should be zero now

In [33]:
df_all_good_indices['yrdx_plus_modx'] = df_all_good_indices.yrdx1.astype('str') + df_all_good_indices.month1.astype('str')


df_all_good_indices['yrdx_plus_modx'] = pd.to_datetime(df_all_good_indices['yrdx_plus_modx'], 
                                                      format='%Y%m')

ValueError: time data '2009nan' does not match format '%Y%m' (match)

In [ ]:
dh = dh.loc[df_all_good_indices.index]

dh['yrdx_plus_modx'] = df_all_good_indices['yrdx_plus_modx']
dh['site1'] = df_all_good_indices['site1']

In [ ]:
dh['medpar_admission_date_new'] = dh.medpar_admission_date.astype('str').str.replace('\.0','')
dh['medpar_admission_date_new'] = dh.medpar_admission_date_new.str.rjust(8,'0')

In [ ]:
dh = dh[~dh['medpar_admission_date_new'].str.contains('nan',case=False)]
dh.shape

In [ ]:
dh['medpar_admission_date_new'] = pd.to_datetime(dh['medpar_admission_date_new'], 
                                                      format='%m%d%Y').copy()

In [ ]:
dh = dh[(dh.medpar_admission_date_new >= dh.yrdx_plus_modx)]
dh.shape

# <font color="#E56717">There are up to 25 + 1 different icd-9-cm dx codes associated with each claim</font>

<span style="font-size:1.5em;">The most common icd9 dx codes in the medpar claims: (raw counts)</span>

In [ ]:
control_code_1 = dh['dgn_cd1_description'].loc[goodcontrolindices].value_counts()
treatment_code_1 = dh['dgn_cd1_description'].loc[goodtreatmentindices].value_counts()

control_code_2 = dh['dgn_cd2_description'].loc[goodcontrolindices].value_counts()
treatment_code_2 = dh['dgn_cd2_description'].loc[goodtreatmentindices].value_counts()

control_code_3 = dh['dgn_cd3_description'].loc[goodcontrolindices].value_counts()
treatment_code_3 = dh['dgn_cd3_description'].loc[goodtreatmentindices].value_counts()

control_code_4 = dh['dgn_cd4_description'].loc[goodcontrolindices].value_counts()
treatment_code_4 = dh['dgn_cd4_description'].loc[goodtreatmentindices].value_counts()


control_code_5 = dh['dgn_cd5_description'].loc[goodcontrolindices].value_counts()
treatment_code_5 = dh['dgn_cd5_description'].loc[goodtreatmentindices].value_counts()


control_code_6 = dh['dgn_cd6_description'].loc[goodcontrolindices].value_counts()
treatment_code_6 = dh['dgn_cd6_description'].loc[goodtreatmentindices].value_counts()


control_code_7 = dh['dgn_cd7_description'].loc[goodcontrolindices].value_counts()
treatment_code_7 = dh['dgn_cd7_description'].loc[goodtreatmentindices].value_counts()


control_code_8 = dh['dgn_cd8_description'].loc[goodcontrolindices].value_counts()
treatment_code_8 = dh['dgn_cd8_description'].loc[goodtreatmentindices].value_counts()



control_code_9 = dh['dgn_cd9_description'].loc[goodcontrolindices].value_counts()
treatment_code_9 = dh['dgn_cd9_description'].loc[goodtreatmentindices].value_counts()


control_code_10 = dh['dgn_cd10_description'].loc[goodcontrolindices].value_counts()
treatment_code_10 = dh['dgn_cd10_description'].loc[goodtreatmentindices].value_counts()

##################################################################################################

control_code_11 = dh['dgn_cd11_description'].loc[goodcontrolindices].value_counts()
treatment_code_11 = dh['dgn_cd11_description'].loc[goodtreatmentindices].value_counts()

control_code_12 = dh['dgn_cd12_description'].loc[goodcontrolindices].value_counts()
treatment_code_12 = dh['dgn_cd12_description'].loc[goodtreatmentindices].value_counts()

control_code_13 = dh['dgn_cd13_description'].loc[goodcontrolindices].value_counts()
treatment_code_13 = dh['dgn_cd13_description'].loc[goodtreatmentindices].value_counts()

control_code_14 = dh['dgn_cd14_description'].loc[goodcontrolindices].value_counts()
treatment_code_14 = dh['dgn_cd14_description'].loc[goodtreatmentindices].value_counts()


control_code_15 = dh['dgn_cd15_description'].loc[goodcontrolindices].value_counts()
treatment_code_15 = dh['dgn_cd15_description'].loc[goodtreatmentindices].value_counts()


control_code_16 = dh['dgn_cd16_description'].loc[goodcontrolindices].value_counts()
treatment_code_16 = dh['dgn_cd16_description'].loc[goodtreatmentindices].value_counts()


control_code_17 = dh['dgn_cd17_description'].loc[goodcontrolindices].value_counts()
treatment_code_17 = dh['dgn_cd17_description'].loc[goodtreatmentindices].value_counts()


control_code_18 = dh['dgn_cd18_description'].loc[goodcontrolindices].value_counts()
treatment_code_18 = dh['dgn_cd18_description'].loc[goodtreatmentindices].value_counts()



control_code_19 = dh['dgn_cd19_description'].loc[goodcontrolindices].value_counts()
treatment_code_19 = dh['dgn_cd19_description'].loc[goodtreatmentindices].value_counts()


control_code_20 = dh['dgn_cd20_description'].loc[goodcontrolindices].value_counts()
treatment_code_20 = dh['dgn_cd20_description'].loc[goodtreatmentindices].value_counts()


control_code_21 = dh['dgn_cd21_description'].loc[goodcontrolindices].value_counts()
treatment_code_21 = dh['dgn_cd21_description'].loc[goodtreatmentindices].value_counts()

control_code_22 = dh['dgn_cd22_description'].loc[goodcontrolindices].value_counts()
treatment_code_22 = dh['dgn_cd22_description'].loc[goodtreatmentindices].value_counts()



control_code_23 = dh['dgn_cd23_description'].loc[goodcontrolindices].value_counts()
treatment_code_23 = dh['dgn_cd23_description'].loc[goodtreatmentindices].value_counts()


control_code_24 = dh['dgn_cd24_description'].loc[goodcontrolindices].value_counts()
treatment_code_24 = dh['dgn_cd24_description'].loc[goodtreatmentindices].value_counts()


control_code_25 = dh['dgn_cd25_description'].loc[goodcontrolindices].value_counts()
treatment_code_25 = dh['dgn_cd25_description'].loc[goodtreatmentindices].value_counts()


control_code_admdxcde = dh['admdxcde_description'].loc[goodcontrolindices].value_counts()
treatment_code_admdxcde = dh['admdxcde_description'].loc[goodtreatmentindices].value_counts()










In [ ]:
dfkodes = pd.DataFrame({'control_code_1': control_code_1,
                        'control_code_2': control_code_2,
                        'control_code_3': control_code_3,
                        'control_code_4': control_code_4,
                        'control_code_5': control_code_5,
                        'control_code_6': control_code_6,
                        'control_code_7': control_code_7,
                        'control_code_8': control_code_8,
                        'control_code_9': control_code_9,
                        'control_code_10': control_code_10,
                        'treatment_code_1': treatment_code_1,
                        'treatment_code_2': treatment_code_2,
                        'treatment_code_3': treatment_code_3,
                        'treatment_code_4': treatment_code_4,
                        'treatment_code_5': treatment_code_5,
                        'treatment_code_6': treatment_code_6,
                        'treatment_code_7': treatment_code_7,
                        'treatment_code_8': treatment_code_8,
                        'treatment_code_9': treatment_code_9,
                        'treatment_code_10': treatment_code_10,
                        'control_code_11': control_code_11,
                        'control_code_12': control_code_12,
                        'control_code_13': control_code_13,
                        'control_code_14': control_code_14,
                        'control_code_15': control_code_15,
                        'control_code_16': control_code_16,
                        'control_code_17': control_code_17,
                        'control_code_18': control_code_18,
                        'control_code_19': control_code_19,
                        'control_code_20': control_code_20,
                        'treatment_code_11': treatment_code_11,
                        'treatment_code_12': treatment_code_12,
                        'treatment_code_13': treatment_code_13,
                        'treatment_code_14': treatment_code_14,
                        'treatment_code_15': treatment_code_15,
                        'treatment_code_16': treatment_code_16,
                        'treatment_code_17': treatment_code_17,
                        'treatment_code_18': treatment_code_18,
                        'treatment_code_19': treatment_code_19,
                        'treatment_code_20': treatment_code_20,
                        'control_code_21': control_code_21,
                        'control_code_22': control_code_22,
                        'control_code_23': control_code_23,
                        'control_code_24': control_code_24,
                        'control_code_25': control_code_25,
                         'treatment_code_21': treatment_code_21,
                        'treatment_code_22': treatment_code_22,
                        'treatment_code_23': treatment_code_23,
                        'treatment_code_24': treatment_code_24,
                        'treatment_code_25': treatment_code_25,
                        'control_code_admdxcde': control_code_admdxcde,
                        'treatment_code_admdxcde': treatment_code_admdxcde
                       })

In [ ]:
dfkodes['control_code'] = dfkodes[['control_code_1',
                                  'control_code_2',
                                  'control_code_3',
                                  'control_code_4',
                                  'control_code_5',
                                  'control_code_6',
                                  'control_code_7',
                                  'control_code_8',
                                  'control_code_9',
                                  'control_code_10',
                                   'control_code_11',
                                  'control_code_12',
                                  'control_code_13',
                                  'control_code_14',
                                  'control_code_15',
                                  'control_code_16',
                                  'control_code_17',
                                  'control_code_18',
                                  'control_code_19',
                                  'control_code_20',
                                    'control_code_21',
                                  'control_code_22',
                                  'control_code_23',
                                  'control_code_24',
                                  'control_code_25',
                                   'control_code_admdxcde'
                                  ]].sum(axis=1)

In [ ]:
dfkodes['treatment_code'] = dfkodes[['treatment_code_1',
                                  'treatment_code_2',
                                  'treatment_code_3',
                                  'treatment_code_4',
                                  'treatment_code_5',
                                  'treatment_code_6',
                                  'treatment_code_7',
                                  'treatment_code_8',
                                  'treatment_code_9',
                                  'treatment_code_10',
                                   'treatment_code_11',
                                  'treatment_code_12',
                                  'treatment_code_13',
                                  'treatment_code_14',
                                  'treatment_code_15',
                                  'treatment_code_16',
                                  'treatment_code_17',
                                  'treatment_code_18',
                                  'treatment_code_19',
                                  'treatment_code_20',
                                    'treatment_code_21',
                                  'treatment_code_22',
                                  'treatment_code_23',
                                  'treatment_code_24',
                                  'treatment_code_25',
                                   'treatment_code_admdxcde'
                                  ]].sum(axis=1)

In [ ]:
dfkodes['difference'] = np.abs(dfkodes.control_code - dfkodes.treatment_code)
dfkodes['together'] = dfkodes.control_code + dfkodes.treatment_code
dfkodes = dfkodes.sort_values(['together','difference'],ascending=False)

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[1:15][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[15:30][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[30:45][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
searchcols = list(dh.columns)[54:79]
searchcols.append(dh.columns[104])
searchcols

In [ ]:
dfkodes['control_code'][dfkodes['control_code'].index.str.contains('atrial',case=False,na=False)]

# <font color="#E56717">Determine the relative likelihood of experiencing certain complications, and whether or not any observed differences are statistically significant</font>

<span style="font-size:1.5em;">For each matched - pair, get an indicator variable for whether or not they had a claim in the medpar file with the particular code of interest anywhere in the potentially 25 + 1 icd9 dx columns. It doesn't matter if it appears in more than one claim: either a 1 or 0.</span>

In [ ]:
def make_filter(pid, df, condition):
    """pid is a patient_id, df is a dataframe, and condition
    is the string. Returns a boolean of whether or not any of the 
    columns searchcols in df.loc[pid] as strings contain condition."""
    
    if pid not in df.index:
        check_dict = dict()
        check_dict[pid] = 0
        return check_dict
    
    dh = df.loc[pid].copy()
    
    if type(dh) == pd.core.series.Series:
        dh = pd.DataFrame(dh).transpose().copy()
        
    
    f0 = dh[searchcols[0]].str.contains(condition,case=False,na=False)
    f1 = dh[searchcols[1]].str.contains(condition,case=False,na=False)
    f2 = dh[searchcols[2]].str.contains(condition,case=False,na=False)
    f3 = dh[searchcols[3]].str.contains(condition,case=False,na=False)
    f4 = dh[searchcols[4]].str.contains(condition,case=False,na=False)
    f5 = dh[searchcols[5]].str.contains(condition,case=False,na=False)
    f6 = dh[searchcols[6]].str.contains(condition,case=False,na=False)
    f7 = dh[searchcols[7]].str.contains(condition,case=False,na=False)
    f8 = dh[searchcols[8]].str.contains(condition,case=False,na=False)
    f9 = dh[searchcols[9]].str.contains(condition,case=False,na=False)
    f10 = dh[searchcols[10]].str.contains(condition,case=False,na=False)
    f11 = dh[searchcols[11]].str.contains(condition,case=False,na=False)
    f12 = dh[searchcols[12]].str.contains(condition,case=False,na=False)
    f13 = dh[searchcols[13]].str.contains(condition,case=False,na=False)
    f14 = dh[searchcols[14]].str.contains(condition,case=False,na=False)
    f15 = dh[searchcols[15]].str.contains(condition,case=False,na=False)
    f16 = dh[searchcols[16]].str.contains(condition,case=False,na=False)
    f17 = dh[searchcols[17]].str.contains(condition,case=False,na=False)
    f18 = dh[searchcols[18]].str.contains(condition,case=False,na=False)
    f19 = dh[searchcols[19]].str.contains(condition,case=False,na=False)
    f20 = dh[searchcols[20]].str.contains(condition,case=False,na=False)
    f21 = dh[searchcols[21]].str.contains(condition,case=False,na=False)
    f22 = dh[searchcols[22]].str.contains(condition,case=False,na=False)
    f23 = dh[searchcols[23]].str.contains(condition,case=False,na=False)
    f24 = dh[searchcols[24]].str.contains(condition,case=False,na=False)
    f25 = dh[searchcols[25]].str.contains(condition,case=False,na=False)
    
    bigcheck = f1 | f2 | f3 | f4 | f5 | f6 | f7 | f8 | f9 | f10 | \
               f11 | f12 | f13 | f14 | f15 | f16 | f17 | f18 | f19 | \
              f20 | f21 | f22 | f23 | f24 | f25
            
    check_dict = dict()
    #print(bigcheck.astype('int'))
    #print(type(bigcheck))
    check_dict[pid] = bigcheck.astype('int').max()
    #print(check_dict)
    return check_dict

In [ ]:
from collections import OrderedDict
chronickidneycontrol = OrderedDict()

for i in range(len(list(goodcontrolindices))):
    little_guy = make_filter(list(goodcontrolindices)[i], dh[searchcols], 'neoplasm of bone')
    chronickidneycontrol[list(goodcontrolindices)[i]] = little_guy[list(goodcontrolindices)[i]]


In [ ]:
from collections import OrderedDict
chronickidneytreatment = OrderedDict()

for i in range(len(list(goodtreatmentindices))):
    little_guy = make_filter(list(goodtreatmentindices)[i], dh[searchcols], 'neoplasm of bone')
    chronickidneytreatment[list(goodtreatmentindices)[i]] = little_guy[list(goodtreatmentindices)[i]]

In [ ]:

q = list(chronickidneycontrol.items())

controlchronickidneyvals = [q[p][1] for p in range(len(list(chronickidneycontrol.items())))]

controlchronickidneyindex = [q[p][0] for p in range(len(list(chronickidneycontrol.items())))]

In [ ]:
q = list(chronickidneytreatment.items())

treatmentchronickidneyvals = [q[p][1] for p in range(len(list(chronickidneytreatment.items())))]

treatmentchronickidneyindex = [q[p][0] for p in range(len(list(chronickidneytreatment.items())))]

In [ ]:
dfchronickidney = pd.DataFrame({'control_vals': controlchronickidneyvals,
                               'control_ids': controlchronickidneyindex,
                               'treatment_vals': treatmentchronickidneyvals,
                               'treatment_ids': treatmentchronickidneyindex})

# <font color="#E56717">Computed an indicator variable for whether or not a given patient had any dx code in the MedPAR file with "neoplasm of bone" as part of the description; collapsed some codes together</font>

In [ ]:
dfchronickidney.describe()

In [ ]:
from scipy.stats import ranksums, wilcoxon

t, p = wilcoxon(dfchronickidney.control_vals.as_matrix(),
               dfchronickidney.treatment_vals.as_matrix())

print(t,p)

# <font color="#E56717">Also test for signficance of the difference in the probability of a "secondary malignant neoplasm of bone and bone marrow" icd9-dx code in a hospital stay using Bayesian A/B testing</font>

In [ ]:
probcontrol = dfchronickidney.control_vals.sum()/913
probtreatment = dfchronickidney.treatment_vals.sum()/913
print(probcontrol,probtreatment)


In [ ]:
controltotal = 854
treatmenttotal = 854
controluninsured = dfchronickidney.control_vals.sum()
treatmentuninsured = dfchronickidney.treatment_vals.sum()

In [ ]:
TREATMENT_UNINSURED = treatmentuninsured
CONTROL_UNINSURED = controluninsured
TREATMENT_TOTAL = treatmenttotal
CONTROL_TOTAL = controltotal
from scipy.stats import beta
alpha_prior = 1
beta_prior = 1
posterior_treatment = beta(alpha_prior + TREATMENT_UNINSURED, beta_prior + TREATMENT_TOTAL - TREATMENT_UNINSURED)
posterior_control = beta(alpha_prior + CONTROL_UNINSURED, beta_prior + CONTROL_TOTAL - CONTROL_UNINSURED)
samples = 2000000
samples_posterior_treatment = posterior_treatment.rvs(samples)
samples_posterior_control = posterior_control.rvs(samples)
(samples_posterior_treatment > samples_posterior_control).mean()

In [ ]:
x = np.linspace(0,1, 50000)
plt.plot(x, posterior_treatment.pdf(x), label='posterior of treatment')
plt.plot(x, posterior_control.pdf(x), label='posterior of control');
plt.xlabel('Value')
plt.ylabel('Density')
plt.title("""Posterior distributions of the treatment and control
cancer patients\n probablities of being hospitalized\n for a bone neoplasm problem""")
plt.legend();
plt.xlim(0,.2)

# Turn all the above into a function

In [ ]:
def make_cf(some_text):
    chronickidneycontrol = OrderedDict()
    for i in range(len(list(goodcontrolindices))):
        little_guy = make_filter(list(goodcontrolindices)[i], dh[searchcols], some_text)
        chronickidneycontrol[list(goodcontrolindices)[i]] = little_guy[list(goodcontrolindices)[i]]
    chronickidneytreatment = OrderedDict()
    for i in range(len(list(goodtreatmentindices))):
        little_guy = make_filter(list(goodtreatmentindices)[i], dh[searchcols], some_text)
        chronickidneytreatment[list(goodtreatmentindices)[i]] = little_guy[list(goodtreatmentindices)[i]]
    q = list(chronickidneycontrol.items())
    controlchronickidneyvals = [q[p][1] for p in range(len(list(chronickidneycontrol.items())))]
    controlchronickidneyindex = [q[p][0] for p in range(len(list(chronickidneycontrol.items())))]
    q = list(chronickidneytreatment.items())
    treatmentchronickidneyvals = [q[p][1] for p in range(len(list(chronickidneytreatment.items())))]
    treatmentchronickidneyindex = [q[p][0] for p in range(len(list(chronickidneytreatment.items())))]
    dfchronickidney = pd.DataFrame({'control_vals': controlchronickidneyvals,
                               'control_ids': controlchronickidneyindex,
                               'treatment_vals': treatmentchronickidneyvals,
                               'treatment_ids': treatmentchronickidneyindex})
    t, p = wilcoxon(dfchronickidney.control_vals.as_matrix(),
               dfchronickidney.treatment_vals.as_matrix())
    probcontrol = dfchronickidney.control_vals.sum()/913
    probtreatment = dfchronickidney.treatment_vals.sum()/913
    print("control prob:" ,probcontrol,"treatment prob:", probtreatment)
    print("p-value from wilcoxon test:", p)
    controltotal = 854
    treatmenttotal = 854
    controluninsured = dfchronickidney.control_vals.sum()
    treatmentuninsured = dfchronickidney.treatment_vals.sum()
    TREATMENT_UNINSURED = treatmentuninsured
    CONTROL_UNINSURED = controluninsured
    TREATMENT_TOTAL = treatmenttotal
    CONTROL_TOTAL = controltotal
    from scipy.stats import beta
    alpha_prior = 1
    beta_prior = 1
    posterior_treatment = beta(alpha_prior + TREATMENT_UNINSURED, beta_prior + TREATMENT_TOTAL - TREATMENT_UNINSURED)
    posterior_control = beta(alpha_prior + CONTROL_UNINSURED, beta_prior + CONTROL_TOTAL - CONTROL_UNINSURED)
    samples = 2000000
    samples_posterior_treatment = posterior_treatment.rvs(samples)
    samples_posterior_control = posterior_control.rvs(samples)
    abtest = (samples_posterior_treatment > samples_posterior_control).mean()
    print("abtest sampling result of treatment > control:", abtest)
    
    x = np.linspace(0,1, 50000)
    plt.plot(x, posterior_treatment.pdf(x), label='posterior of treatment')
    plt.plot(x, posterior_control.pdf(x), label='posterior of control');
    plt.xlabel('Value')
    plt.ylabel('Density')
    atitle = "probabilties for being hospitalized for a " + some_text + " problem"
    plt.title(atitle)
    plt.legend();
    plt.xlim(0,.3)

In [ ]:
make_cf('neoplasm of bone')

In [ ]:
make_cf('kidney failure')

In [ ]:
make_cf('airway obstruction')

In [ ]:
make_cf('anemia')

In [ ]:
make_cf('urinary tract infection')

In [ ]:
make_cf('syncope')

In [ ]:
make_cf('hypopotassemia')

In [ ]:
make_cf('dehydration')

# Toxic Effects of Radiation codes

In [ ]:
make_cf('Irradiation cystitis')

In [ ]:
make_cf('Overdose of radiation in therapy')

In [ ]:
dfkodes['difference'] = np.abs(dfkodes.control_code - dfkodes.treatment_code)
dfkodes['together'] = dfkodes.control_code + dfkodes.treatment_code
dfkodes = dfkodes.sort_values(['together'],ascending=False)

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[1:15][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
make_cf('heart failure')

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[15:30][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
make_cf('hypertensive chronic kidney disease')

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[45:60][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
make_cf('malaise and fatigue')

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[45:60][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
make_cf('vascular disease')

In [ ]:
dfkodes[['control_code','treatment_code']].iloc[60:75][::-1].plot(kind='barh',
                title="Diagnosis codes top values in MedPAR claims");

In [ ]:
make_cf('hemorrhage of gastrointestinal tract')

In [ ]:
make_cf('chronic kidney disease')